You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan Document:

Title: Exploring the Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Importance of staying updated on the latest trends and key players in the AI landscape

Key Points:
1. Latest Trends in Artificial Intelligence:
- Advancements in machine learning algorithms
- Growth of AI-powered chatbots and virtual assistants
- Increased use of AI in healthcare, finance, and marketing indu

I now can give a great answer
Final Answer: 
# Exploring the Latest Trends and Key Players in Artificial Intelligence

## Introduction

Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing the way we work and interact with technology. From enhancing customer experiences to optimizing business operations, AI continues to push the boundaries of innovation. Staying updated on the latest trends and key players in the AI landscape is crucial for businesses and individuals looking to leverage the power of AI effectively.

## Latest Trends in Artificial Intelligence

Advancements in machine learning algorithms have propelled AI to new heights, enabling more accurate predictions and personalized recommendations. The growth of AI-powered chatbots and virtual assistants has streamlined customer service processes and improved user engagement. Moreover, industries such as healthcare, finance, and marketing are increasingly embracing AI to drive 

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Exploring the Latest Trends and Key Players in Artificial Intelligence

## Introduction

Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing the way we work and interact with technology. From enhancing customer experiences to optimizing business operations, AI continues to push the boundaries of innovation. Staying updated on the latest trends and key players in the AI landscape is crucial for businesses and individuals looking to leverage the power of AI effectively.

## Latest Trends in Artificial Intelligence

Advancements in machine learning algorithms have propelled AI to new heights, enabling more accurate predictions and personalized recommendations. The growth of AI-powered chatbots and virtual assistants has streamlined customer service processes and improved user engagement. Moreover, industries such as healthcare, finance, and marketing are increasingly embracing AI to drive efficiency and gain a competitive edge. However, with these advancements come ethical considerations in AI development, raising concerns about data privacy and bias in algorithmic decision-making.

## Key Players in Artificial Intelligence

Leading the charge in AI innovation are key players such as Google's DeepMind, IBM Watson, Microsoft Azure AI, Amazon Web Services AI, OpenAI, and NVIDIA. These companies are at the forefront of developing cutting-edge AI solutions that are shaping the future of technology. By investing in research and partnerships, they are driving progress in AI capabilities and setting new standards for industry best practices.

## Noteworthy News in Artificial Intelligence

Recent partnerships and acquisitions in the AI industry have sparked collaborations that are accelerating the pace of innovation. Breakthrough research and developments in AI technology, such as advancements in natural language processing and computer vision, are pushing the boundaries of what AI can achieve. Regulatory updates impacting the use of AI in different sectors are also shaping the future of AI governance and accountability, ensuring that ethical principles are integrated into AI systems.

## Conclusion

As AI continues to evolve and disrupt traditional business models, staying informed about the latest trends and key players in the industry is essential for navigating the rapidly changing landscape. By keeping abreast of AI developments and understanding the impact of AI on various sectors, businesses and individuals can harness the full potential of AI technology. Subscribe to our newsletter or follow our social media accounts for regular updates on AI trends and news, and join us at upcoming AI conferences or webinars to deepen your knowledge on this transformative technology.

Remember, the future of AI is in your hands. Stay informed, stay ahead.

*This blog post was crafted with insights from Forbes AI articles, TechCrunch AI section, MIT Technology Review AI coverage, AI conferences and events calendar, and Gartner AI research reports.*

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "Mughal Dynasty"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Mughal Dynasty.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan Document:

Title: Unveiling the Magnificence of the Mughal Dynasty

Introduction:
- Brief overview of the Mughal Dynasty, highlighting its significance in Indian history.
- Mention of key players such as Babur, Akbar, and Shah Jahan.
- Teaser about the key points to be discussed in the article.

Key Points:
1. Historical Background of the Mughal Dynasty:
   - Origin and establishment of the dynasty by Babur in 1526.
   - Expansion and consolidation under Akbar's rule.
   - Architectural wonders like the T

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).